# **1. Basic Setup**

In [ ]:
#cloning repository
!git clone https://github.com/WadRex/Maskifier.git

In [ ]:
import urllib.request

#downloading .weights file
urllib.request.urlretrieve('https://github.com/WadRex/Maskifier/releases/download/v1.0/yolov4_head.weights', 'Maskifier/YOLO/yolov4_head.weights')

In [ ]:
pip install -r '/content/Maskifier/requirements.txt'

# **2. Webcam & YOLO Initialization**

In [ ]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

In [ ]:
import cv2
import numpy as np
from yolov4.tf import YOLOv4
from google.colab.patches import cv2_imshow

#yolo for head detection
headyolo = YOLOv4()
headyolo.config.parse_names('/content/Maskifier/YOLO/yolov4_head.names')
headyolo.config.parse_cfg('/content/Maskifier/YOLO/yolov4_head.cfg')
headyolo.make_model()
headyolo.load_weights('/content/Maskifier/YOLO/yolov4_head.weights', weights_type='yolo')
headyolo.summary(summary_type='yolo')
headyolo.summary()

# **3.a) Single Detection**

In [ ]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

In [ ]:
#where everything happens
from IPython.display import Image
try:
  filename = take_photo()
except Exception as err:
  # Errors will be thrown if the user does not have a webcam or if they do not
  # grant the page permission to access it.
  print(str(err))
 
#get image 
cam = cv2.imread('/content/photo.jpg')

#get mask detection
mask_pred_bboxes = headyolo.predict(cam, 0.5)        

#preprocess mask detection
height, width, _ = cam.shape
mask_bboxes = mask_pred_bboxes * np.array([width, height, width, height, 1, 1])

#draw detections
for bbox in mask_bboxes:
  #check if probability not NULL
  if float(bbox[5]) > 0:
  #define coordinates
    y = int(bbox[1]) - int(bbox[3]) / int(2)
    x = int(bbox[0]) - int(bbox[2]) / int(2)
    h = int(bbox[3])
    w = int(bbox[2]) 

    #normalize?
    if y < 0:
      y = 0
    if x < 0:
      x = 0    

    #find masks
    #not wearing mask
    if int(bbox[4]) == 0:
      print ("not nice | confidence: ", bbox[5])              
      cv2.rectangle(cam, (int(x),int(y)), (int(x+w),int(y+h)), (0,0,255), 10)                                           

    #wearing mask
    if int(bbox[4]) == 1:
      print ("nice | confidence: ", bbox[5])
      cv2.rectangle(cam, (int(x),int(y)), (int(x+w),int(y+h)), (0,255,0), 10)        

#show results
cv2_imshow(cam)
if cv2.waitKey(1) & 0xFF == ord("q"):
  cv2.destroyAllWindows()                             

# **3.b) Continuous Detection**

In [ ]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      'await new Promise((resolve) => capture.onclick = resolve);'

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

In [ ]:
while True:
  #where everything happens
  from IPython.display import Image
  try:
    filename = take_photo()
  except Exception as err:
    # Errors will be thrown if the user does not have a webcam or if they do not
    # grant the page permission to access it.
    print(str(err))
 
  #get image 
  cam = cv2.imread('/content/photo.jpg')

  #get mask detection
  mask_pred_bboxes = headyolo.predict(cam, 0.5)        

  #preprocess mask detection
  height, width, _ = cam.shape
  mask_bboxes = mask_pred_bboxes * np.array([width, height, width, height, 1, 1])

  #draw detections
  for bbox in mask_bboxes:
    #check if probability not NULL
    if float(bbox[5]) > 0:
    #define coordinates
      y = int(bbox[1]) - int(bbox[3]) / int(2)
      x = int(bbox[0]) - int(bbox[2]) / int(2)
      h = int(bbox[3])
      w = int(bbox[2]) 

      #normalize?
      if y < 0:
        y = 0
      if x < 0:
        x = 0    

      #find masks
      #not wearing mask
      if int(bbox[4]) == 0:
        print ("not nice | confidence: ", bbox[5])              
        cv2.rectangle(cam, (int(x),int(y)), (int(x+w),int(y+h)), (0,0,255), 10)                                           

      #wearing mask
      if int(bbox[4]) == 1:
        print ("nice | confidence: ", bbox[5])
        cv2.rectangle(cam, (int(x),int(y)), (int(x+w),int(y+h)), (0,255,0), 10)        

  #show results
  import time
  cv2_imshow(cam)
  time.sleep(1)
  if cv2.waitKey(1) & 0xFF == ord("q"):
    cv2.destroyAllWindows()                             